In [10]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 14.1 MB/s eta 0:00:0000:0100:01


In [11]:
k = 10
alpha = 5

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import faiss

import os
import gc
import torch
from tqdm import tqdm

In [13]:
gpu_ids = "0"  # can be e.g. "3,4" for multiple GPUs 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_ids
faiss.get_num_gpus()

1

In [14]:
class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k
        
    def fit(self, X, y):
        self.cpu_index = faiss.IndexFlatL2(X.shape[1])
        self.gpu_index = faiss.index_cpu_to_all_gpus(self.cpu_index)
        self.gpu_index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X, alpha):
        distances, indices = self.gpu_index.search(X.astype(np.float32), k=self.k, )
        predictions = np.array(self.y[indices])
        w = np.exp(-alpha*np.square(distances))
        return np.sum(w*predictions, axis=1)/np.sum(w, axis=1)

In [15]:
train_df = pd.read_csv("/kaggle/input/aml-dataset/train.csv")

In [ ]:
train_Y = train_df['PRODUCT_LENGTH'].values
train_X = np.load("/kaggle/input/amlc-title-embeddings-vanilla/title_embeddings.npy")
train_Y_log = np.log(train_Y+1)

In [ ]:
model = FaissKNeighbors(k=k)
model.fit(train_X, train_Y_log)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
test_X = np.load("/kaggle/input/amlc-test-title-embeddings/title_embeddings_test.npy")

In [ ]:
predictions = []

for test_x in tqdm(np.split(test_X, 16)):
    y_preds = model.predict(test_x, alpha)
    predictions.append(y_preds)
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
y_preds = np.concatenate(predictions)
y_preds = np.exp(y_preds)-1

In [ ]:
submission = test_df = pd.read_csv("/kaggle/input/aml-dataset/sample_submission.csv")
submission['PRODUCT_LENGTH'] = y_preds
submission.to_csv("KNN_submission.csv")